### GPU CHECKS, IMPORTING LIBRARIES AND DATA

All work was done in AWS, so google colab related code is removed as well as downloading of N-HPatches etc.

In [0]:
%cd keras_triplet_descriptor

!pip install humanize
!pip install talos
!pip install opencv-python
import humanize
import os
  
import sys
import json
import os
import glob
import time
import tensorflow as tf
import numpy as np
import cv2
import random
import talos
import pandas as pd
import matplotlib.pyplot as plt

import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda, Reshape
from keras.layers import Conv2D, Conv3D, MaxPooling2D, BatchNormalization 
from keras.layers import Input, UpSampling2D, concatenate  
from keras.activations import sigmoid, relu, elu
from keras.layers.convolutional import Deconvolution2D
from talos import live, Reporting
from talos.model.normalizers import lr_normalizer
from keras.losses import mean_squared_error, binary_crossentropy

#!git clone https://github.com/MatchLab-Imperial/keras_triplet_descriptor

#!wget -O hpatches_data.zip https://imperialcollegelondon.box.com/shared/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip
  
#!unzip -q ./hpatches_data.zip
#!rm ./hpatches_data.zip

hpatches_dir = './hpatches'
splits_path = './splits.json'

splits_json = json.load(open(splits_path, 'rb'))
split = splits_json['a']

train_fnames = split['train']
test_fnames = split['test']

seqs = glob.glob(hpatches_dir+'/*')
seqs = [os.path.abspath(p) for p in seqs]   
seqs_train = list(filter(lambda x: x.split('/')[-1] in train_fnames, seqs)) 
seqs_test = list(filter(lambda x: x.split('/')[-1] in split['test'], seqs))

from read_data import HPatches, DataGeneratorDesc, hpatches_sequence_folder, DenoiseHPatches, tps
from utils import generate_desc_csv, plot_denoise, plot_triplet

random.seed(1234)
np.random.seed(1234)
tf.set_random_seed(1234)

### TESTING VARIOUS DENOISING ARCHITECTURES

In [0]:
denoise_generator = DenoiseHPatches(random.sample(seqs_train, 3), batch_size=50)
denoise_generator_val = DenoiseHPatches(random.sample(seqs_test, 1), batch_size=50)

100%|██████████| 1/1 [00:00<00:00,  1.19it/s]


Baseline architecture defined.

In [0]:
def get_denoise_model_baseline(shape):
 
  inputs = Input(shape)
  

  conv1 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
  
  conv2 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)

  up3 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv2))
  merge3 = concatenate([conv1,up3], axis = -1)
  conv3 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge3)
    
  conv4 = Conv2D(1, 3,  padding = 'same')(conv3)

  shallow_net = Model(inputs = inputs, outputs = conv4)
  
  return shallow_net

get_denoise_model_baseline((32, 32, 1)).summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   160         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 16, 16, 16)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 16, 16, 32)   4640        max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
up_samplin

Deeper UNet proposed architecture is defined.

In [0]:
def get_denoise_model_deeperUNet(shape):
  
  inputs = Input(shape)
  
  ## Encoder part
  conv1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
  conv2 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
  pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
  conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
  pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
  conv4 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
  drop4 = Dropout(rate = 0.5)(conv4)
  pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

  conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
  drop5 = Dropout(rate = 0.5)(conv5)
  ## Now the decoder starts
  up6 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
  merge6 = concatenate([drop4,up6], axis = 3)
  conv6 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)

  up7 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
  merge7 = concatenate([conv3,up7], axis = 3)
  conv7 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)

  up8 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
  merge8 = concatenate([conv2,up8], axis = 3)
  conv8 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)

  up9 = Conv2D(32, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
  merge9 = concatenate([conv1,up9], axis = 3)
  conv9 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
  conv10 = Conv2D(1, 3,  padding = 'same')(conv9)

  model = Model(inputs = inputs, outputs = conv10)
  
  return model

get_denoise_model_deeperUNet((32, 32, 1)).summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 1)    0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 32, 32, 32)   320         input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D)  (None, 16, 16, 32)   0           conv2d_6[0][0]                   
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 16, 16, 64)   18496       max_pooling2d_2[0][0]            
__________________________________________________________________________________________________
max_poolin

Autoencoder proposed architecture is defined.

In [0]:
def get_denoise_model_autoencoder(shape):
  
  model = Sequential()

  ## Encoder starts
  model.add(Conv2D(filters=32, kernel_size=(3,3), strides=(2,2), input_shape = (32, 32, 1), padding='same', activation='relu'))
  model.add(Conv2D(filters=64, kernel_size=(3,3), strides=(2,2), padding='same', activation='relu'))
  model.add(Conv2D(filters=128, kernel_size=(3,3), strides=(2,2), padding='same', activation='relu'))

  ## And now the decoder
  model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))
  model.add(UpSampling2D((2, 2)))
  model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu'))
  model.add(UpSampling2D((2, 2)))
  model.add(Conv2D(filters=3, kernel_size=(3,3), padding='same', activation='relu'))
  model.add(UpSampling2D((2, 2)))

  # Improve smoothness of the image:
  model.add(Conv2D(filters=1, kernel_size=(3,3), padding='same', activation='relu'))
  
  return model

get_denoise_model_autoencoder((32, 32, 1)).summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 16, 16, 32)        320       
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 8, 8, 64)          18496     
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 4, 4, 128)         73856     
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 4, 4, 64)          73792     
_________________________________________________________________
up_sampling2d_6 (UpSampling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 8, 8, 32)          18464     
_________________________________________________________________
up_sampling2d_7 (UpSampling2 (None, 16, 16, 32)        0         
__________

Compiling all models with the same hyperparameters and then training all of them for 25 epochs.

In [0]:
keras.backend.clear_session()

shape = (32, 32, 1)
denoise_model_baseline = get_denoise_model_baseline(shape)
denoise_model_deeperUNet = get_denoise_model_deeperUNet(shape)
denoise_model_autoencoder = get_denoise_model_autoencoder(shape)

sgd = keras.optimizers.SGD(lr=0.00001, momentum=0.9, nesterov=True)

denoise_model_baseline.compile(loss='mean_absolute_error', optimizer=sgd, metrics=['mae'])
denoise_model_deeperUNet.compile(loss='mean_absolute_error', optimizer=sgd, metrics=['mae'])
denoise_model_autoencoder.compile(loss='mean_absolute_error', optimizer=sgd, metrics=['mae'])

In [0]:
denoise_history_baseline = denoise_model_baseline.fit_generator(generator=denoise_generator, epochs=25, verbose=1, validation_data=denoise_generator_val)

In [0]:
denoise_history_deeperUNet = denoise_model_deeperUNet.fit_generator(generator=denoise_generator, epochs=25, verbose=1, validation_data=denoise_generator_val)

In [0]:
denoise_history_autoencoder = denoise_model_autoencoder.fit_generator(generator=denoise_generator, epochs=25, verbose=1, validation_data=denoise_generator_val)

Visualising the results.

In [0]:
fig, axes = plt.subplots(3,1, figsize = (21, 3))
p1 = axes[0].plot(denoise_history_baseline.history['loss'], 'r')
p2 = axes[0].plot(denoise_history_baseline.history['val_loss'], 'g')
axes[0].set_title('Baseline De-noise Model Training Loss')
axes[0].set_ylabel('Error')
axes[0].set_xlabel('Number of epochs')
axes[0].legend((p1[0], p2[0]), ('Training Loss', 'Test Loss'))

p3 = axes[1].plot(denoise_history_deeperUNet.history['loss'], 'r')
p4 = axes[1].plot(denoise_history_deeperUNet.history['val_loss'], 'g')
axes[1].set_title('Deeper UNet De-noise Model Training Loss')
axes[1].set_ylabel('Error')
axes[1].set_xlabel('Number of epochs')
axes[1].legend((p3[0], p4[0]), ('Training Loss', 'Test Loss'))

p5 = axes[2].plot(denoise_history_autoencoder.history['loss'], 'r')
p6 = axes[2].plot(denoise_history_autoencoder.history['val_loss'], 'g')
axes[2].set_title('Autoencoder Model Training Loss')
axes[2].set_ylabel('Error')
axes[2].set_xlabel('Number of epochs')
axes[2].legend((p5[0], p6[0]), ('Training Loss', 'Test Loss'))

Horizontal bar graph of network performance results.

In [0]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(figsize = (7, 1))

# Example data
people = ("Baseline", "Deeper UNet", "Autoencoder")
performance = (6.65, 6.33 ,8.24)

ax.barh(people, performance, color='green', ecolor='black')
ax.set_yticks(y_pos)
ax.set_yticklabels(people)
ax.invert_yaxis() 
ax.set_xlabel('Loss')
ax.set_title('Architectures Loss')

plt.show()

In [0]:
print("Baseline plots:")
plot_denoise(denoise_model_baseline)

In [0]:
print("Deeper UNet plots:")
plot_denoise(denoise_model_deeperUNet)

In [0]:
print("Autoencoder plots:")
plot_denoise(denoise_model_autoencoder)